In [1]:
import os
import playsound
import speech_recognition as sr
import time
import sys
import ctypes
import wikipedia
import datetime
import json
import re
import webbrowser
import smtplib
import requests
import urllib
import urllib.request as urllib2
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from time import strftime
from gtts import gTTS
from youtube_search import YoutubeSearch


In [2]:
wikipedia.set_lang('vi')
language = 'vi'
path = ChromeDriverManager().install()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 91.0.4472
[WDM] - Get LATEST driver version for 91.0.4472
[WDM] - Driver [C:\Users\ACER\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [3]:
def speak(text):
    print("Bot: {}".format(text))
    tts = gTTS(text=text, lang=language, slow=False)
    tts.save("sound.mp3")
    playsound.playsound("sound.mp3", False)
    os.remove("sound.mp3")

In [4]:
def get_audio():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Tôi: ", end='')
        audio = r.listen(source, phrase_time_limit=12)
        try:
            text = r.recognize_google(audio, language="vi-VN")
            print(text)
            return text
        except:
            print("...")
            return 0

In [5]:
def stop():
    speak("Muộn rồi. Đi ngủ đi!")

In [6]:
def get_text():
    for i in range(3):
        text = get_audio()
        if text:
            return text.lower()
        elif i < 2:
            time.sleep(5)
            speak("Tôi không nghe rõ. Bạn nói lại được không!")
    time.sleep(15)
    stop()
    return 0

In [7]:
def hello(name):
    day_time = int(strftime('%H'))
    if 4 <day_time < 8:
        speak("Chào buổi sáng bạn {}. Chúc bạn một ngày tốt lành.".format(name))
    elif 8 <= day_time < 12:
        speak("{} ơi. Cô kêu xuống ăn cơm trưa".format(name))
    else:
        speak("{} ơi. Cô kêu xuống ăn cơm tối".format(name))

In [8]:
def get_time(text):
    now = datetime.datetime.now()
    if "giờ" in text:
        speak('Bây giờ là %d giờ %d phút' % (now.hour, now.minute))
    elif "ngày" in text:
        speak("Hôm nay là ngày %d tháng %d năm %d" %
              (now.day, now.month, now.year))
    else:
        speak("Bot chưa hiểu ý của bạn. Bạn nói lại được không?")

In [9]:
def open_application(text):
    if "google" in text:
        speak("Mở Google Chrome")
        os.startfile('C:\Program Files (x86)\Google\Chrome\Application\chrome.exe')
    elif "word" in text:
        speak("Mở Microsoft Word")
        os.startfile('C:\Program Files\Microsoft Office\\root\Office16\\WINWORD.EXE')
    elif "excel" in text:
        speak("Mở Microsoft Excel")
        os.startfile('C:\Program Files\Microsoft Office\\root\Office16\EXCEL.EXE')
    else:
        speak("Ứng dụng chưa được cài đặt. Bạn hãy thử lại!")

In [10]:
def open_website(text):
    reg_ex = re.search('mở (.+)', text)
    if reg_ex:
        domain = reg_ex.group(1)
        url = 'https://www.' + domain
        webbrowser.open(url)
        speak("Trang web bạn yêu cầu đã được mở.")
        return True
    else:
        return False

In [11]:
def open_google_and_search(text):
    search_for = text.split("kiếm", 1)[1]
    speak('Okay!')
    driver = webdriver.Chrome(path)
    driver.get("http://www.google.com")
    que = driver.find_element_by_xpath("//input[@name='q']")
    que.send_keys(str(search_for))
    que.send_keys(Keys.RETURN)

In [12]:
# def send_email(text):
#     speak('Bạn gửi email cho ai nhỉ')
#     recipient = get_text()
#     if 'yến' in recipient:
#         speak('Nội dung bạn muốn gửi là gì')
#         content = get_text()
#         mail = smtplib.SMTP('smtp.gmail.com', 587)
#         mail.ehlo()
#         mail.starttls()
#         mail.login('luongngochungcntt@gmail.com', 'hung23081997')
#         mail.sendmail('luongngochungcntt@gmail.com',
#                       'hungdhv97@gmail.com', content.encode('utf-8'))
#         mail.close()
#         speak('Email của bạn vùa được gửi. Bạn check lại email nhé hihi.')
#     else:
#         speak('Bot không hiểu bạn muốn gửi email cho ai. Bạn nói lại được không?')

In [13]:
def current_weather():
    speak("Bạn muốn xem thời tiết ở đâu ạ.")
    ow_url = "http://api.openweathermap.org/data/2.5/weather?"
    city = get_text()
    if not city:
        pass
    api_key = "fe8d8c65cf345889139d8e545f57819a"
    call_url = ow_url + "appid=" + api_key + "&q=" + city + "&units=metric"
    response = requests.get(call_url)
    data = response.json()
    if data["cod"] != "404":
        city_res = data["main"]
        current_temperature = city_res["temp"]
        current_pressure = city_res["pressure"]
        current_humidity = city_res["humidity"]
        suntime = data["sys"]
        sunrise = datetime.datetime.fromtimestamp(suntime["sunrise"])
        sunset = datetime.datetime.fromtimestamp(suntime["sunset"])
        wthr = data["weather"]
        weather_description = wthr[0]["description"]
        now = datetime.datetime.now()
        content = """
        Hôm nay là ngày {day} tháng {month} năm {year}
        Mặt trời mọc vào {hourrise} giờ {minrise} phút
        Mặt trời lặn vào {hourset} giờ {minset} phút
        Nhiệt độ trung bình là {temp} độ C
        Áp suất không khí là {pressure} héc tơ Pascal
        Độ ẩm là {humidity}%
        Trời hôm nay quang mây. Dự báo mưa rải rác ở một số nơi.""".format(day = now.day,month = now.month, year= now.year, hourrise = sunrise.hour, minrise = sunrise.minute,
                                                                           hourset = sunset.hour, minset = sunset.minute, 
                                                                           temp = current_temperature, pressure = current_pressure, humidity = current_humidity)
        speak(content)
        time.sleep(20)
    else:
        speak("Không tìm thấy địa chỉ của bạn")

In [14]:
def play_song():
    speak('Xin mời bạn chọn tên bài hát')
    mysong = get_text()
    while True:
        result = YoutubeSearch(mysong, max_results=10).to_dict()
        if result:
            break

    url = 'https://www.youtube.com' + result[0]['channel/UCLLgZpTl5qReeqd9i8btZxQ']
    webbrowser.open(url)
    speak("Bài hát bạn yêu cầu đã được mở.")

In [15]:
# def change_wallpaper():
#     api_key = 'RF3LyUUIyogjCpQwlf-zjzCf1JdvRwb--SLV6iCzOxw'
#     url = 'https://api.unsplash.com/photos/random?client_id=' + \
#         api_key  # pic from unspalsh.com
#     f = urllib2.urlopen(url)
#     json_string = f.read()
#     f.close()
#     parsed_json = json.loads(json_string)
#     photo = parsed_json['urls']['full']
#     # Location where we download the image to.
#     urllib2.urlretrieve(photo, "C:/Users/Night Fury/Downloads/a.png")
#     image=os.path.join("C:/Users/Night Fury/Downloads/a.png")
#     ctypes.windll.user32.SystemParametersInfoW(20,0,image,3)
#     speak('Hình nền máy tính vừa được thay đổi')

In [16]:
def read_news():
    speak("Bạn muốn đọc báo về gì")
    
    queue = get_text()
    params = {
        'apiKey': '30d02d187f7140faacf9ccd27a1441ad',
        "q": queue,
    }
    api_result = requests.get('http://newsapi.org/v2/top-headlines?', params)
    api_response = api_result.json()
    print("Tin tức")

    for number, result in enumerate(api_response['articles'], start=1):
        print(f"""Tin {number}:\nTiêu đề: {result['title']}\nTrích dẫn: {result['description']}\nLink: {result['url']}
    """)
        if number <= 3:
            webbrowser.open(result['url'])

In [17]:
def tell_me_about():
    try:
        speak("Bạn muốn nghe về gì ạ")
        text = get_text()
        contents = wikipedia.summary(text).split('\n')
        speak(contents[0])
        time.sleep(60)
        for content in contents[1:]:
            speak("Bạn muốn nghe thêm không")
            ans = get_text()
            if "có" or "nghe"not in ans:
                break    
            speak(content)
            time.sleep(60)

        time.sleep(60)
        speak('Cảm ơn bạn đã lắng nghe!!!')
    except:
        speak("Tôi không định nghĩa được thuật ngữ của bạn. Xin mời bạn nói lại")

In [18]:
def help_me():
    speak("""Bot có thể giúp bạn thực hiện các câu lệnh sau đây:
    1. Chào hỏi
    2. Hiển thị giờ
    3. Mở website, application
    4. Tìm kiếm trên Google
    5. Gửi email
    6. Dự báo thời tiết
    7. Mở video nhạc
    8. Thay đổi hình nền máy tính
    9. Đọc báo hôm nay
    10. Kể bạn biết về thế giới """)

In [22]:
def assistant():
    speak("Xin chào, bạn tên là gì?")
    name = get_text()
    if name:
        speak("Chào bạn {}".format(name))
        time.sleep(5)
        speak("Tôi có thể giúp gì cho bạn?")
        time.sleep(5)
        while True:
            text = get_text()
            if not text:
                break
            elif "dừng" in text or "tạm biệt" in text or "chào robot" in text or "ngủ thôi" in text:
                stop()
                break
            elif "giúp tôi với" in text:
                help_me()
                time.sleep(5)
            elif "xem giờ" in text:
                get_time(text)
                time.sleep(5)
            elif "xin" in text:
                hello(name)
                time.sleep(5)
            elif "tìm kiếm" in text:
                if 'mở google và tìm kiếm' in text:
                    open_google_and_search(text)
                elif "." in text:
                    open_website(text)
                else:
                    open_application(text)
            elif "email" in text or "mail" in text or "gmail" in text:
                send_email(text)
                time.sleep(5)
            elif "thời tiết" in text:
                current_weather()
                time.sleep(5)
            elif "nghe nhạc" in text:
                play_song()
                time.sleep(5)
            elif "hình nền" in text:
                change_wallpaper()
                time.sleep(5)
            elif "đọc báo" in text:
                read_news()
                time.sleep(5)
            elif "định nghĩa" in text:
                tell_me_about()
                time.sleep(5)
            else:
                speak("Bạn cần tôi giúp gì ạ?")

In [ ]:
 assistant()

Bot: Xin chào, bạn tên là gì?
Tôi: ...
Bot: Tôi không nghe rõ. Bạn nói lại được không!
Tôi: ...
